In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install laserembeddings
!python -m laserembeddings download-models

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from laserembeddings import Laser

import re
from tqdm.auto import tqdm
tqdm.pandas()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.5 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=6964f168d93fa0d07bd92c85d3f6a03179b16f795df114907bb30b304d5080fe
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.53
    Uninstalling sacremoses-0.0.53:
      Successfully uninstalled sacremoses-0.0.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.

✅   Downloaded https://dl.fbaipublicfiles.com/laser/mod

In [2]:
en_bukhari_muslim = pd.read_csv('../input/en-bn-sahih-bukhari-muslim/en_bn_translated_sahih_hadiths_final.csv')
en_bukhari_muslim.head(1)

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation - كتاب بدء الوحى,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...,'Abdullah bin al-Zubair bin 'Isa عبد الله بن ا...,উমর বিন আল-খাততাবের কাহিনীঃ আমি আল্লার রসূলকে ...


In [3]:
en_bukhari_muslim.narrators[0]

"'Abdullah bin al-Zubair bin 'Isa عبد الله بن الزبير بن عيسى الحميدي,Sufyan bin 'Uyaynah سفيان بن عيينة,Yahya bin Sa'id al-Ansari يحيى بن سعيد الأنصاري,Muhammad bin Ibrahim bin al-Harith محمد بن إبراهيم بن الحارث,'Alqama bin Waqqas علقمة بن وقاص,'Umar ibn al-Khattab ( عمر بن الخطاب بن نفيل ( رضي الله عنه"

In [4]:
%%time

def remove_non_english(mixed_text):
    return re.sub("[^a-zA-Z0-9,']+", " ",str(mixed_text))


en_bukhari_muslim["narrators"]=en_bukhari_muslim.narrators.progress_apply(lambda mixed_text: remove_non_english(mixed_text))
en_bukhari_muslim["chapter"]=en_bukhari_muslim.chapter.progress_apply(lambda mixed_text: remove_non_english(mixed_text))
en_bukhari_muslim.narrators[0]

  0%|          | 0/14966 [00:00<?, ?it/s]

  0%|          | 0/14966 [00:00<?, ?it/s]

CPU times: user 340 ms, sys: 6.32 ms, total: 346 ms
Wall time: 343 ms


"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin 'Uyaynah ,Yahya bin Sa'id al Ansari ,Muhammad bin Ibrahim bin al Harith ,'Alqama bin Waqqas ,'Umar ibn al Khattab "

In [5]:
en_bukhari_muslim.chapter[0]

'Revelation '

In [6]:
%%time
laser = Laser()


en_bukhari_muslim['merged_text'] = en_bukhari_muslim.text_en.astype(str) + en_bukhari_muslim.chapter.astype(str)

embeds = laser.embed_sentences(en_bukhari_muslim['merged_text'].values, lang='en')
np.save("en_emb_bukhari_muslim.npy",embeds)
en_bukhari_muslim = en_bukhari_muslim.drop(columns=['merged_text'])

CPU times: user 1min 6s, sys: 7.52 s, total: 1min 13s
Wall time: 1min 15s


In [7]:
en_bukhari_muslim.head()

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...,"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin '...",উমর বিন আল-খাততাবের কাহিনীঃ আমি আল্লার রসূলকে ...
1,1,2,Sahih Bukhari,1,2,Revelation,"30355, 20001, 11065, 10511, 53",حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هش...,Narrated 'Aisha: ...,"'Abdullah bin Yusuf al Tunaysi ,Imam Maalik ,H...","আয়েশা (রাঃ) এর কাহিনী বর্ণিত হয়েছে, তিনি বলে..."
2,2,3,Sahih Bukhari,1,3,Revelation,"30399, 20023, 11207, 11013, 10511, 53",حدثنا يحيى بن بكير، قال حدثنا الليث، عن عقيل، ...,Narrated 'Aisha: (the m...,"Yahya bin Yahya bin Bukayr ,al Laith bin Sa'd ...",'আয়েশা (রাঃ) এর বানী বর্ণিত হয়েছেঃ (বিশ্বাসী...
3,3,4,Sahih Bukhari,1,4,Revelation,"11013, 10567, 34",قال ابن شهاب وأخبرني أبو سلمة بن عبد الرحمن، أ...,Narrated Jabir bin 'Abdullah Al-Ansari while ...,"al Zuhri , ,Abu Salama bin 'Abdur Rahman ,Jabi...",জাবির বিন 'আব্দুল্লাহ আল-আনসারি বর্ণনা করেছেন ...
4,4,5,Sahih Bukhari,1,5,Revelation,"20040, 20469, 11399, 11050, 17",حدثنا موسى بن إسماعيل، قال حدثنا أبو عوانة، قا...,Narrated Said bin Jubair: ...,"Musa bin Isma'il al Munqari al Tabudhaki ,Wada...","তিনি বলেন, ইবনে জুবাইর বলেনঃ ইবনে আব্বাস আল্লা..."


In [8]:
df = en_bukhari_muslim[en_bukhari_muslim['text_en'].str.contains('not found yet')]
len(df)

0

In [9]:
df = en_bukhari_muslim[en_bukhari_muslim['text_ar'].str.contains('not found yet')]
len(df)

6

In [10]:
df = en_bukhari_muslim[en_bukhari_muslim['text_bn'].str.contains('এখনো পাওয়া যায়নি')]
len(df)

0

In [11]:
en_bukhari_muslim.text_en[100]

' Narrated Abu Sa`id Al-Khudri:                     Some women requested the Prophet to fix a day for them as the men were taking all his time. On that he promised them one day for religious lessons and commandments. Once during such a lesson the Prophet said, "A woman whose three children die will be shielded by them from the Hell fire." On that a woman asked, "If only two die?" He replied, "Even two (will shield her from the Hell-fire).'

In [12]:
en_bukhari_muslim.to_csv('en_bukhari_muslim.csv',index = False)

we are using [this dataset](https://www.kaggle.com/datasets/mobassir/en-bn-sahih-bukhari-muslim) which was scrapped from muslimscholars.info & qaalarasulallah.com database by [Fahd](https://www.kaggle.com/fahd09) that can be found here -> https://www.kaggle.com/datasets/fahd09/hadith-dataset and here -> https://www.kaggle.com/datasets/fahd09/hadith-narrators
we pick only bukhari and muslim hadith's from that english source and add one new column that contains the list of "narrators".
data processing code can be found here -> https://github.com/mnansary/hadith-srcapper/blob/main/simple_EDA/simple-eda-between-en-bn-scrapped-hadiths.ipynb

also we've replaced 531 missing english and bangla hadith human translation with machine translation from arabic source as shown here -> https://github.com/mobassir94/Multilingual-NLP-for-Islamic-Theology/blob/main/demo_notebooks/ar-en-and-ar-bn-missing-hadith-translator.ipynb